# SQL and Python: Alerts from time series predictions
- query data in Tinybird using SQL
- model time-series data using Prophet (python)
- write predictions to Tinybird
- compare the predictions to actual data
- alert in Tinybird when actual data is out of predicted limits using SQL
- plot actual against predicted

In [1]:
#@title Install the Tinybird CLI, libraries and your token
!pip install tinybird-cli -q -U

import os
import io
import pandas as pd

from fbprophet import Prophet
import matplotlib.pyplot as plt
from google.colab import files

!tb init
!tb auth

     |████████████████████████████████| 75 kB 753 kB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 86 kB 4.4 MB/s 
     |████████████████████████████████| 81 kB 8.0 MB/s 
     |████████████████████████████████| 61 kB 7.5 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.25.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
** - /datasources created
** - /datasources/fixtures created
** - /endpoints created
** - /explorations created
** - /pipes created
Copy the admin token from https://ui.tinybird.co/tokens and paste it here: 
** Auth successful! 
** Configuration written to .tinyb file, consider adding it to .gitign

In [2]:
#@title Helper function
def write_text_to_file(filename, text):
  with open(filename, 'w') as f: f.write(text)

This notebook uses two Data Sources:
- taxi for 2018 and 2019 data (for modelling)
- taxi_2020_01 for January 2020 data (to compare to the predictions)

There is a guide to ingesting the New York City taxi data to Tinybird [here](https://github.com/tinybirdco/RTACourse/blob/main/New_York_City_Taxi_Trip_Datasets.ipynb).

In [3]:
#@title Query data in Tinybird using SQL and create a pandas DataFrame
output=!tb sql --rows_limit 1000 --format csv \
"SELECT toDate(tpep_pickup_datetime) as date, \
count() as events FROM taxi \
WHERE date BETWEEN '2018-01-01' AND '2019-12-31' \
GROUP BY date"

In [4]:
df=pd.read_csv(io.StringIO(output.n))
print(len(df), 'rows in DataFrame')

730 rows in DataFrame


In [5]:
df.head()

,date,events
0,2018-01-01,237119
1,2018-01-02,238152
2,2018-01-03,266992
3,2018-01-04,122222
4,2018-01-05,265212


In [6]:
df.tail()

,date,events
725,2019-12-27,168061
726,2019-12-28,171034
727,2019-12-29,163808
728,2019-12-30,172965
729,2019-12-31,167615


In [7]:
#@title Model time-series data using Prophet (python) and generate predictions
days_to_predict = 31

prediction_df = pd.DataFrame()
prediction_df['ds'] = df.date
prediction_df['y'] = df.events

model = Prophet(daily_seasonality=True, 
                weekly_seasonality=True, 
                yearly_seasonality=True)
model.fit(prediction_df)

future = model.make_future_dataframe(periods=days_to_predict)
forecast = model.predict(future)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [8]:
#@title Plot the historic data and the predictions for the next month
fig = model.plot(forecast)
fig.savefig('prophet_plot_1.jpg', dpi=600)
files.download('prophet_plot_1.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
#@title Plot the components of the model
fig = model.plot_components(forecast)
fig.savefig('prophet_plot_2.jpg', dpi=600)
files.download('prophet_plot_2.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
#@title Write the predictions to a Tinybird Data Source

for col in ['yhat_lower','yhat','yhat_upper']:
  forecast[col] = forecast[col].astype('int64')

forecast[['ds', 'yhat_lower', 'yhat', 'yhat_upper']].to_csv('./datasources/fixtures/forecast.csv', index=False)
!tb datasource generate ./datasources/fixtures/forecast.csv --force

** Generated datasources/forecast.datasource
** => Create it on the server running: $ tb push datasources/forecast.datasource
** => Append data using: $ tb datasource append forecast ./datasources/fixtures/forecast.csv`

** => Generated fixture datasources/fixtures/forecast.None


In [11]:
!cat datasources/forecast.datasource

DESCRIPTION generated from ./datasources/fixtures/forecast.csv

SCHEMA >
    `ds` Date,
    `yhat_lower` Int64,
    `yhat` Int64,
    `yhat_upper` Int64

In [12]:
!tb datasource rm forecast --yes

** Data Source 'forecast' deleted


In [13]:
!tb push datasources/forecast.datasource --fixtures  

** Processing datasources/forecast.datasource
** Building dependencies
** Running forecast 
** 'forecast' created
** Pushing fixtures
** Checking forecast.datasource (appending 23.8 kb)
**  OK (1 blocks)


In [14]:
#@title Compare the predictions to the actual data for January 2020 in Tinybird using a Pipe and an Endpoint.
filename="pipes/actual_vs_expected.pipe"
text='''
DESCRIPTION >
    comparison of actual vs expected 

NODE actual_events
DESCRIPTION >
    events per day in January 2020
SQL >

    SELECT toDate(tpep_pickup_datetime) as ds, count() as events 
    FROM taxi_2020_01 
    WHERE ds BETWEEN '2020-01-01' AND '2020-01-31' 
    GROUP BY ds

NODE with_predicted_events
DESCRIPTION >
    join with predicted events for January 2020
SQL >

    SELECT * FROM actual_events
    LEFT JOIN forecast
    USING ds

NODE beyond_the_limits
DESCRIPTION >
    identify anomalous days
SQL >

    SELECT *,
    If(events < yhat_lower, 'error', 'OK') AS anomaly_lower,
    If(events > yhat_upper, 'error', 'OK') AS anomaly_higher
    FROM with_predicted_events
'''

write_text_to_file(filename, text)

In [15]:
!tb push 'pipes/actual_vs_expected.pipe' --force --no-check

** Processing pipes/actual_vs_expected.pipe
** Warning: taxi_2020_01 not found inside: 
   - /content
   - /content/datasources
   - /content/endpoints
** Building dependencies
** Running actual_vs_expected 
** => Test endpoint at https://api.tinybird.co/v0/pipes/actual_vs_expected.json
** 'actual_vs_expected' created
** Not pushing fixtures


In [16]:
#@title The same results from clicking on the test Endpoint can be created in a pandas DataFrame.
output = !tb sql --format csv "SELECT toDate(tpep_pickup_datetime) as date, count() as events \
FROM taxi_2020_01 GROUP BY date"

forecast = forecast.set_index('ds')
df_actual = pd.read_csv(io.StringIO(output.n)).set_index('date')
df_actual = df_actual[(df_actual.index > '2020-01-00') & (df_actual.index < '2020-02-00')]
df_actual.index = pd.to_datetime(df_actual.index)
df_actual = df_actual.join(forecast[['yhat_lower', 'yhat', 'yhat_upper']])
df_actual['anomaly_lower'] = 'OK'
df_actual['anomaly_higher'] = 'OK'
df_actual.loc[df_actual.events < df_actual.yhat_lower, 'anomaly_lower'] = 'error'
df_actual.loc[df_actual.events > df_actual.yhat_upper, 'anomaly_higher'] = 'error'
df_actual

,events,yhat_lower,yhat,yhat_upper,anomaly_lower,anomaly_higher
date,,,,,,
2020-01-01,168578,143240,168549,195869,OK,OK
2020-01-02,160291,147768,175573,202600,OK,OK
2020-01-03,181520,149409,175105,202002,OK,OK
2020-01-04,181706,134633,160139,187196,OK,OK
2020-01-05,163038,100460,126310,154295,OK,error
2020-01-06,175535,108928,137655,163196,OK,error
2020-01-07,196964,135519,162965,190971,OK,error
2020-01-08,212584,147135,173420,200307,OK,error
2020-01-09,227500,158833,185033,212670,OK,error


In [17]:
#@title Plot the predicted values with error bars and compare to the actual values
ax = plt.subplot(111)
plt.bar(df_actual.index, df_actual.yhat, alpha=.4)
plt.errorbar(df_actual.index, df_actual.yhat, 
             yerr=(df_actual.yhat_upper-df_actual.yhat_lower)/2, 
             fmt="o", color="r", alpha=.3)
plt.scatter(df_actual.index,df_actual.events, color="k")
plt.xticks(rotation=-45)
plt.title('Actual taxi trips compared to predicted taxi trips')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.tight_layout()
plt.savefig('actual_preds.jpg', dpi=600)
files.download('actual_preds.jpg');

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
#@title Alert: lower number of trips than expected
df_actual[df_actual.events < df_actual.yhat_lower]

,events,yhat_lower,yhat,yhat_upper,anomaly_lower,anomaly_higher
date,,,,,,
2020-01-20,161069,162043,189735,215080,error,OK


In [19]:
#@title Alert: higher number of trips than expected
df_actual[df_actual.events > df_actual.yhat_upper]

,events,yhat_lower,yhat,yhat_upper,anomaly_lower,anomaly_higher
date,,,,,,
2020-01-05,163038,100460,126310,154295,OK,error
2020-01-06,175535,108928,137655,163196,OK,error
2020-01-07,196964,135519,162965,190971,OK,error
2020-01-08,212584,147135,173420,200307,OK,error
2020-01-09,227500,158833,185033,212670,OK,error
2020-01-10,224085,161491,188860,216142,OK,error
2020-01-11,215631,149792,177812,203347,OK,error
2020-01-12,184056,121210,147452,175563,OK,error
2020-01-13,207317,135909,161754,190848,OK,error
